<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/proyecto_apoyo/02_RecuperacionInformacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TF-IDF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Base de conocimiento
data = [
    "Resetear contraseña", "Encontrar número de serie", "Dispositivo no enciende"
]
vectorizer = TfidfVectorizer()
vector_matrix = vectorizer.fit_transform(data)

# Consulta del usuario
query = "olvidé mi contraseña"
#query = "olvidé mi clave"
query_vec = vectorizer.transform([query])
similarities = cosine_similarity(query_vec, vector_matrix).flatten()

# Mostrar resultados
ranking = similarities.argsort()[::-1]
for idx in ranking:
    print(f"Documento: {data[idx]} | Similitud: {similarities[idx]:.2f}")


Documento: Resetear contraseña | Similitud: 0.71
Documento: Dispositivo no enciende | Similitud: 0.00
Documento: Encontrar número de serie | Similitud: 0.00


#Embeddings

In [ ]:
from sentence_transformers import SentenceTransformer, util


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Base de conocimiento
data = [
    "Resetear contraseña", "Encontrar número de serie", "Dispositivo no enciende"
]
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
data_embeddings = model.encode(data, convert_to_tensor=True)

# Consulta del usuario
#query = "olvidé mi contraseña"
query = "olvidé mi clave"
query_embedding = model.encode(query, convert_to_tensor=True)
similarities = util.pytorch_cos_sim(query_embedding, data_embeddings).squeeze()

# Mostrar resultados
ranking = similarities.argsort(descending=True)
for idx in ranking:
    print(f"Documento: {data[idx]} | Similitud: {similarities[idx]:.2f}")

Documento: Resetear contraseña | Similitud: 0.55
Documento: Dispositivo no enciende | Similitud: 0.32
Documento: Encontrar número de serie | Similitud: 0.11


#Ejemplo de comparación temporal

##Calculando embeddings

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util

# Cargar un subset del dataset de Wikipedia en español
dataset = load_dataset("wikimedia/wikipedia", "20231101.es", split="train")
data = [item["text"] for item in dataset.select(range(20000)) if isinstance(item["text"], str)]

# Consulta del usuario
query = "¿Cuál es la banda más importante de España?"

# **TF-IDF**
vectorizer = TfidfVectorizer()
start_time_tfidf = time.time()
vector_matrix = vectorizer.fit_transform(data)
query_vec = vectorizer.transform([query])
tfidf_similarities = cosine_similarity(query_vec, vector_matrix).flatten()
end_time_tfidf = time.time()

# Ordenar resultados TF-IDF
ranking_tfidf = tfidf_similarities.argsort()[::-1]
top_tfidf = [(data[idx], tfidf_similarities[idx], dataset['title'][idx]) for idx in ranking_tfidf[:5]]

# Embeddings
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")  # Modelo multilingüe para español
start_time_embeddings = time.time()
data_embeddings = model.encode(data, convert_to_tensor=True)
query_embedding = model.encode(query, convert_to_tensor=True)
embedding_similarities = util.pytorch_cos_sim(query_embedding, data_embeddings).squeeze().cpu().numpy()
end_time_embeddings = time.time()

# Ordenar resultados Embeddings
ranking_embeddings = embedding_similarities.argsort()[::-1]
top_embeddings = [(data[idx], embedding_similarities[idx],dataset['title'][idx]) for idx in ranking_embeddings[:5]]

# Mostrar resultados
print("=== Resultados TF-IDF ===")
print(f"Tiempo de cálculo: {end_time_tfidf - start_time_tfidf:.4f} segundos")
for idx, (doc, sim, tit) in enumerate(top_tfidf):
    print(f"\nDocumento {idx+1}:")
    print(f"Similitud: {sim:.4f}\n{tit}\n{doc[:300]}")

print("\n=== Resultados Embeddings ===")
print(f"Tiempo de cálculo: {end_time_embeddings - start_time_embeddings:.4f} segundos")
for idx, (doc, sim,tit) in enumerate(top_embeddings):
    print(f"\nDocumento {idx+1}:")
    print(f"Similitud: {sim:.4f}\n{tit}\n{doc[:300]}")



=== Resultados TF-IDF ===
Tiempo de cálculo: 46.0444 segundos

Documento 1:
Similitud: 0.3878
Banda
Banda

Grupo de personas 
 Banda, grupo étnico de la República Centroafricana, que constituye aproximadamente un 40% de la población.
 Sociedad de bandas, unión de clanes que dejan de ser nómadas y se convierten en sedentarios. 
 Sinónimo de pandilla, grupo de personas.

Música 

 Banda de música, e

Documento 2:
Similitud: 0.3162
Banda base
En Telecomunicaciones, el término banda base se refiere a la banda de frecuencias producida por un transductor, tal como un micrófono, un manipulador telegráfico u otro dispositivo generador de señales que no es necesario adaptarlo al medio por el que se va a transmitir.

Banda base es la señal de u

Documento 3:
Similitud: 0.2682
Banda ancha
En telecomunicaciones, se conoce como banda ancha a cualquier tipo de red con elevada capacidad para transportar información que incide en la velocidad de transmisión de esta. Así entonces, es la transmisión de 

##Cargando embeddings pre-calculados

Ahora guardo los modelos, embeddings y vectores para agilizar el proceso.

In [ ]:
#Guardo los embeddings y vectores ya generados anteriormente
import pickle

with open('data_embeddings.pkl', 'wb') as f:
    pickle.dump(data_embeddings, f)

with open('vector_matrix.pkl', 'wb') as f:
    pickle.dump(vector_matrix, f)

with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

#guardo el modelo
model.save('model')


In [ ]:
#Guardo los datos (data y dataset)
import pickle

with open('data.pkl', 'wb') as f:
    pickle.dump(data, f)

with open('dataset.pkl', 'wb') as f:
    pickle.dump(dataset, f)

In [ ]:
from datasets import load_dataset
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util

# Cargar los datos
with open('data.pkl', 'rb') as f:
    data = pickle.load(f)
with open('dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)

# Consulta del usuario
query = "¿Cuál es la banda más importante de España?"

# **TF-IDF**
# Cargo vectorizador y vectores
with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)
start_time_tfidf = time.time()
with open('vector_matrix.pkl', 'rb') as f:
    vector_matrix = pickle.load(f)
query_vec = vectorizer.transform([query])
tfidf_similarities = cosine_similarity(query_vec, vector_matrix).flatten()
end_time_tfidf = time.time()

# Ordenar resultados TF-IDF
ranking_tfidf = tfidf_similarities.argsort()[::-1]
top_tfidf = [(data[idx], tfidf_similarities[idx], dataset['title'][idx]) for idx in ranking_tfidf[:5]]

# **Embeddings**
#cargo modelo y embeddings
model = SentenceTransformer('model')
start_time_embeddings = time.time()
with open('data_embeddings.pkl', 'rb') as f:
    data_embeddings = pickle.load(f)
query_embedding = model.encode(query, convert_to_tensor=True)
embedding_similarities = util.pytorch_cos_sim(query_embedding, data_embeddings).squeeze().cpu().numpy()
end_time_embeddings = time.time()

# Ordenar resultados Embeddings
ranking_embeddings = embedding_similarities.argsort()[::-1]
top_embeddings = [(data[idx], embedding_similarities[idx],dataset['title'][idx]) for idx in ranking_embeddings[:5]]

# Mostrar resultados
print("=== Resultados TF-IDF ===")
print(f"Tiempo de cálculo: {end_time_tfidf - start_time_tfidf:.4f} segundos")
for idx, (doc, sim, tit) in enumerate(top_tfidf):
    print(f"\nDocumento {idx+1}:")
    print(f"Similitud: {sim:.4f}\n{tit}\n{doc[:300]}")

print("\n=== Resultados Embeddings ===")
print(f"Tiempo de cálculo: {end_time_embeddings - start_time_embeddings:.4f} segundos")
for idx, (doc, sim,tit) in enumerate(top_embeddings):
    print(f"\nDocumento {idx+1}:")
    print(f"Similitud: {sim:.4f}\n{tit}\n{doc[:300]}")

=== Resultados TF-IDF ===
Tiempo de cálculo: 1.6608 segundos

Documento 1:
Similitud: 0.3878
Banda
Banda

Grupo de personas 
 Banda, grupo étnico de la República Centroafricana, que constituye aproximadamente un 40% de la población.
 Sociedad de bandas, unión de clanes que dejan de ser nómadas y se convierten en sedentarios. 
 Sinónimo de pandilla, grupo de personas.

Música 

 Banda de música, e

Documento 2:
Similitud: 0.3162
Banda base
En Telecomunicaciones, el término banda base se refiere a la banda de frecuencias producida por un transductor, tal como un micrófono, un manipulador telegráfico u otro dispositivo generador de señales que no es necesario adaptarlo al medio por el que se va a transmitir.

Banda base es la señal de u

Documento 3:
Similitud: 0.2682
Banda ancha
En telecomunicaciones, se conoce como banda ancha a cualquier tipo de red con elevada capacidad para transportar información que incide en la velocidad de transmisión de esta. Así entonces, es la transmisión de d